<style>
.jp-RenderedHTMLCommon pre {
        background-color: #FFFFF0FF;
}
    
.reveal pre code {
	padding: 1px;
	overflow: auto;
	max-height: 95%;
	word-wrap: normal;
    font-size: 0.9em;
    background-color: #FFFFF0FF
}
</style>

# Distributed Systems
## 2022/22

Lab 3

Nuno Preguiça, Sérgio Duarte, João Resende

# Goals

In the end of this lab you should be able to:

+ Know how to deal with errors on REST clients;

## REST - Client Errors

## Causes

A REST request might fail for several reasons:

+ The server is not running;
+ The server is slow;
+ A TCP connection was dropped;
+ The network failed;
+ There was a network anomaly (e.g., routing)


## Transient failures

Temporary failures can be ***masked*** by issuing
the request multiple times.

Usually, the client quits after a few retries to
avoid blocking the application forever,<br>for example, in
case the server has crashed.

Note:

Transient failures are temporary issues that resolve themselves shortly.

## ProcessingException

Jersey (JAX-RS) exposes request failures to clients in the form of a Java exception: [javax.ws.rs.ProcessingException](https://docs.oracle.com/javaee/7/api/javax/ws/rs/ProcessingException.html)

A **try{ } catch{}** block can be used to retry the request automatically after a small amount of time.


Note: Waiting a bit before retrying the request prevents a too agressive client behavior and allows some time for the transient error condition disappear.

### Example - CreateUser (1)

Setting the client timeout values...

```java
protected static final int READ_TIMEOUT = 5000;
protected static final int CONNECT_TIMEOUT = 5000;

ClientConfig config = new ClientConfig();

config.property(ClientProperties.READ_TIMEOUT, READ_TIMEOUT);
config.property( ClientProperties.CONNECT_TIMEOUT, CONNECT_TIMEOUT);
		
Client client = ClientBuilder.newClient(config);

```

In the future, other changes to client behavior will be done the someway.

### Example - CreateUser (2)
```java
protected static final int MAX_RETRIES = 10;
protected static final int RETRY_SLEEP = 1000;

@Override
public String createUser(User user) {

	WebTarget target = client.target( serverURI ).path( UsersService.PATH );
	for (int i = 0; i < MAX_RETRIES; i++)
		try {
			Response r = target.request()
				.accept(MediaType.APPLICATION_JSON)
				.post(Entity.entity(user, MediaType.APPLICATION_JSON));

            if( r.getStatus() == Status.OK.getStatusCode() && r.hasEntity() )
                // SUCCESS
                return r.readEntity(String.class);
            else {
                System.out.println("Error, HTTP error status: " + r.getStatus() );
                break;
            }
		} catch (ProcessingException x) {
            sleep( RETRY_SLEEP );
        }
	return null; // Report failure
}
```

### Can we do better?

The sample code above needs to be **repeated for all operations** of all services!

 + Doable, but **error prone**, because it invites a lot of ***cut & paste***...

Can we make it more general?

Of course!!!

### Step 1 - Implement the request as a private method

```java 

private String clt_createUser(User user) {
    Response r = target.request()
                    .accept(MediaType.APPLICATION_JSON)
                    .post(Entity.entity(user, MediaType.APPLICATION_JSON));

    if( r.getStatus() == Status.OK.getStatusCode() && r.hasEntity() )
        return r.readEntity(String.class);
    else
        return null;
}
```

### Step 2 - Implement the retry behavior as a generic operation

```java
protected <T> T reTry(java.util.function.Supplier<T> func) {
    for (int i = 0; i < MAX_RETRIES; i++)
        try {
            return func.get(); // Success
        } catch (ProcessingException x) {
			sleep(RETRY_SLEEP);
		} catch (Exception x) {
            // Handle other errors
            break;
        }   
    return null; // Failure
}
```

Note:
    
This method can be part of super class inherited by all service clients.

### Step 3 - Implement the requests with retry behavior

```java
public String createUser(User user) {
    return reTry( () -> clt_createUser( user ));
}

public User getUser(User user, String password) {
    return reTry( () -> clt_getUser( user, password ));
}

```

We are making use of [Java Lambda Expressions](https://docs.oracle.com/javase/tutorial/java/javaOO/lambdaexpressions.html)...

`() -> clt_createUser( user )` is a function that returns a result, making it compatible with the functional interface [Supplier&lt;T&gt;](https://docs.oracle.com/javase/8/docs/api/java/util/function/Supplier.html) used as the parameter of the `reTry` generic method.



# Exercises

1. Test the retry mechanism to mask transient failures;
2. Complete the provided client-side code of UserService service. 

## Test masking transient failures


1. Download this [lab's project](https://preguica.github.io/sd2223/praticas2223/aula3/sd2223-aula3.zip);
2. Build the Docker image, using the usual maven command;

  `mvn clean compile assembly:single docker:build`
3. Launch the server;

 `docker network create -d bridge sdnet` (if necessary)
 
 `docker run --rm -h users-1 --name users-1 --network sdnet -p 8080:8080 sd2223-aula3-xxxxx-yyyyy`
 
4. Create the client container:

    `docker run -it --network sdnet sd2223-aula3-xxxxx-yyyyy /bin/bash`

5. Create a new user.

    In the client container shell, type:

    `java -cp sd2223.jar aula3.clients.CreateUserClient http://users-1:8080/rest nmp 12345 nova "Nuno Preguica"`
    
    Confirm the request succeeded and returned: `nmp`
    
6. Stop the server.

    CTRL-C or execute in another terminal:
    
    `docker rm -f users-1`
    
7. Execute step 5 again.

   This time, since the server is not running, the client should output:
   
   `FINE: ProcessingException: java.net.UnknownHostException: users-1`
   
8. Launch the server again (step 3)

   The client should finish and return: `nmp`
   